# Tourist Advisor
Tool that helps with the Tourist on the trip plan.  Takes the City to be visited as the input.  Finds the list of popular places to visit in the City.  Chart out a day to day itinerary.

In [ ]:
import requests
import pandas as pd
pd.set_option('display.max_rows', 500)

!conda install -c conda-forge geocoder -y
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

print('Libraries imported.')

In [ ]:
CLIENT_ID = 'LSC31VKD4CEXLCRBGKZXS1ZGXYVWG2ANHWRG0XKS1WEPBZQG' # your Foursquare ID
CLIENT_SECRET = '11RZFCKAHBOEKJCWFJHDQALNAIJFNBCURFYL5MZAAP40PUXA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Step 0:  Input the City being visited and the number of days of stay

In [ ]:
address = 'Bangalore, India'
duration = 3   # 3 days

### Step 1:  Using geocoders Nominatim MAP API find the latitude and longitude of the City

In [ ]:
geolocator = Nominatim()
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(lat, lng))

### Step 2: Using Foursqaure Venues/Categories API get the list of supported venue categories.

In [ ]:
cat_url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
cat_result = requests.get(cat_url).json()

### Step 3:  Filter tourist attractive categories from the response

In [ ]:
tourism_categories = ['Theme Park', 'Park', 'National Park', 'Botanical Garden', 'Museum', 'Palace', 'Temple', 
                      'Aquarium', 'Planetarium', 'Zoo', 'Monument / Landmark', 'Capitol Building', 'Spiritual Center']  # Only a limited list is taken for this demo

In [ ]:
fs_tourism_cat_list = []
fs_tourism_cat_id = []
for c1 in cat_result['response']['categories']:
    if c1['name'] in tourism_categories:
        fs_tourism_cat_list.append([c1['name'],c1['id']])
        fs_tourism_cat_id.append(c1['id'])
    for c2 in c1['categories']:
        if c2['name'] in tourism_categories:
                fs_tourism_cat_list.append([c2['name'],c2['id']])
                fs_tourism_cat_id.append(c2['id'])
        if (len(c2['categories']) != 0):
            for c3 in c2['categories']:
                if c3['name'] in tourism_categories:
                    fs_tourism_cat_list.append([c3['name'],c3['id']])
                    fs_tourism_cat_id.append(c3['id'])
                

### Step 3:  List the tourism categories, so that the user can select the categories of interested to him

#### This step is ommitted in this implementation.  Instead all the filtered tourism categories are used.

### Step 4: Using the Foursqaure Venues/explore API get the list of Venues matching the tourism categories.

In [ ]:
# Parameters for the foursquare APIs
LIMIT=100
radius = 500000  # 500km

In [ ]:
results = []
for index, cid in enumerate(fs_tourism_cat_id):
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            cid,
            radius, 
            LIMIT)
    results.append(requests.get(url).json())

#### Step 5: Convert the response received in the json format to Dataframe with columns, Venue Name, ID, Category, Latitude and Longitude. 

In [ ]:
venues_list=[]
for res in results:
    if (len(res['response'])  != 0):
        for v in res['response']['venues']:
            # return only relevant information for each nearby venue
            if (len(v['categories']) != 0):
                venues_list.append((
                    v['name'],         
                    v['location']['lat'], 
                    v['location']['lng'],
                    v['categories'][0]['name'],
                    v['id']))

In [ ]:
travel_venues = pd.DataFrame(venues_list)
travel_venues.columns = ['Name', 'Latitude', 'Longtitude', 'Category', 'Id']

### Step 6: Using the Foursquare Venues/VenueID API get the details of each travel venue.  

### Step 7: Add the columns 'Rating' and 'Likes' to the dataframe based on the values received from the Venue details

### Step 8: Sort the dataframe based on the Rating and Likes.  Pick up top N travel venues.  

##### The above steps are ommitted for this implementation.  As they require multiple calls to the venue/venueID api and venue/venueID calls are limitted 50 per day in Sandbox account. Hence the top travel venues are picked up based on the below list.

In [ ]:
top_travel_venues = ['Cubbon Park', 'Lalbagh Botanical Garden', 'Bangalore Fort', "Tipu's Summer Palace", 'Bangalore Palace', 'Vidhana Soudha', 'Government Museum', 
                    'Heritage Centre & Aerospace Museum', 'Visvesvaraya Industrial and Technological Museum', 'Jawaharlal Nehru Planetarium', 'Bannerghatta National Park', 'Butterfly Park',
                    'Bangalore Aquarium', 'Wonder la Amusement Park', 'Innovative Film City', 'ISKCON Bangalore', 'Bull Temple', 'Art of Living International Center']

In [ ]:
top_venues = pd.DataFrame(columns=['Name', 'Category', 'Id', 'Latitude', 'Longtitude'])
for index, v in travel_venues.iterrows():
    if v['Name'] in top_travel_venues:
        top_venues = top_venues.append(v, ignore_index=True)
top_venues.drop_duplicates('Name', keep='first', inplace=True)
top_venues.reset_index(inplace=True)
top_venues

### Step 9: Form a new dataframe by dropping Id, Category, Rating and Likes columns.

In [ ]:
top_venues_clustering = top_venues.drop(['index', 'Name', 'Category', 'Id'], 1)
top_venues_clustering

### Step 10:  Use K-Means clustering to segment venues based on the Latitude and Longitude values

In [ ]:
# set number of clusters
kclusters = duration

# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, n_init = 12).fit(top_venues_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:
venues_grouped = top_venues.drop(['index', 'Id'], axis=1)

# add clustering labels
venues_grouped['Cluster Labels'] = kmeans.labels_
venues_grouped 

### Step 11:  Display the list of venues (Itinerary for each day) 

#### Day1 itinerary

In [ ]:
venues_grouped.loc[venues_grouped['Cluster Labels'] == 0, ['Name', 'Category']]

#### Day2 itinerary

In [ ]:
venues_grouped.loc[venues_grouped['Cluster Labels'] == 1, ['Name', 'Category']]

#### Day3 itinerary

In [ ]:
venues_grouped.loc[venues_grouped['Cluster Labels'] == 2, ['Name', 'Category']]

### Step 12: Use Folium to visualize the travel venues on the City map. 
#### (Day1 = Red , Day2 = Purple, Day3 = Green)

In [ ]:
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_grouped['Latitude'], venues_grouped['Longtitude'], venues_grouped['Name'], venues_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters